In [1]:
import numpy as np
import pandas as pd
import time
import torch
import hydra
import torch.nn as nn
import torch.nn.functional as functional
from torch.distributions import MultivariateNormal, Independent, Normal, Categorical

import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf, DictConfig
import KFModels as KFM
import tqdm.auto as tqdm

import matplotlib as plt
import nonamortised_models as models

import Data_Generation as DG


#### Initialize hydra and savings:

In [2]:

# Initialize OmegaConf
initialize(config_path="configurations")

# Compose the configuration
cfg = compose(config_name='conf1.yaml')

# Access nested configurations using keys
data_cfg = cfg['data']
theta_training_cfg = cfg['theta_training']
phi_training_cfg = cfg['phi_training']

# Now you can access specific attributes within these nested configurations
device = cfg.device
seed = cfg.seed
save_models = cfg.save_models
save_data = cfg.save_data

print("Device:", device)
print("Seed:", seed)
print("Save Models:", save_models)
print("Save Data:", save_data)

# Print the nested configurations
print("Data Config:", OmegaConf.to_yaml(data_cfg))
print("Theta Training Config:", OmegaConf.to_yaml(theta_training_cfg))


Device: cpu
Seed: None
Save Models: True
Save Data: True
Data Config: path_to_data: null
dim: 10
F_min_eigval: 0.5
F_max_eigval: 1
G_min_eigval: 0.5
G_max_eigval: 1
diagFG: true
U_std: 0.1
V_std: 0.25
num_data: 50000
matrices_to_learn: FG

Theta Training Config: func_type: kernel
theta_lr: 0.03
theta_lr_decay_type: robbins-monro
num_steps_theta_lr_oom_drop: 5000
robbins_monro_theta_lr_decay_rate: 0.6
robbins_monro_theta_lr_decay_bias: 10
theta_minibatch_size: 100
theta_updates_start_T: 5
matrices_to_learn: FG
window_size: 1
num_times_save_data: 100
KRR_lambda: 0.0001
KRR_sigma: 2
KRR_train_sigma: true
KRR_train_lam: false
KRR_init_sigma_median: true
approx_decay: 1.0
approx_with_filter: true
kernel_batch_size: 100
train_kernel_iters: 1
train_kernel_minibatch_size: 100
train_kernel_dataset_size: 100
train_kernel_lr: 0.001



/var/folders/x4/sqc7r28j1qggd73l6jk91xnr0000gn/T/ipykernel_49966/2254655681.py:2: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="configurations")
/Users/gianluca.palmari/anaconda3/envs/online_learning_env/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'conf1.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/Users/gianluca.palmari/anaconda3/envs/online_learning_env/lib/python3.9/site-packages/hydra/core/default_element.py:124: UserWarning: In 'theta_training/kernel1.yaml': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/Users/gianluca.palmari/anaconda3/envs/online_

In [3]:
cfg.keys()

dict_keys(['data', 'phi_training', 'theta_training', 'name', 'device', 'seed', 'save_models', 'save_data'])

In [4]:
NOTEBOOK_MODE = True

# Generate and print seed
seed = np.random.randint(0, 9999999) if cfg.seed is None else cfg.seed
print("seed", seed)

# Seed numpy and torch
np.random.seed(seed)
torch.manual_seed(seed)

# Create saved_models folder if required
saved_models_folder_name = 'saved_models'
if cfg.save_models and not NOTEBOOK_MODE:
    if not os.path.exists(saved_models_folder_name):  
        os.mkdir(saved_models_folder_name)  
elif cfg.save_models and NOTEBOOK_MODE:
    if not os.path.exists(saved_models_folder_name): 
        os.mkdir(saved_models_folder_name)  

seed 3553697


In [5]:
# Create saved_data folder if required
saved_data_folder_name = 'saved_data'
if cfg.save_data and not NOTEBOOK_MODE:
    if not os.path.exists(saved_data_folder_name):  
        os.mkdir(saved_data_folder_name)  
elif cfg.save_data and NOTEBOOK_MODE:
    if not os.path.exists(saved_data_folder_name): 
        os.mkdir(saved_data_folder_name)  

# Function to save numpy array
def save_np(name, x):
    if NOTEBOOK_MODE:
        np.save(f'saved_data/{name}', x)

#### $\underline{Utils:}$

In [6]:
class TimeStore(nn.Module):
    """
        Acts like a ParameterList/ModuleList that just keeps getting longer but
        under the hood it only stores the most recent N values.
        Does not support removing items from the list.
    """
    def __init__(self, first_val, N, type):
        super().__init__()

        if type == "ParameterList":
            self.list = nn.ParameterList([first_val] if first_val else [])
        elif type == "ModuleList":
            self.list = nn.ModuleList([first_val] if first_val else [])
        else:
            raise ValueError("TimeStore unknown type: " + type)
        self.len = 1 if first_val else 0
        self.N = N

    def append(self, val):
        self.list.append(val)
        self.len += 1
        if len(self.list) > self.N:
            self.list = self.list[-self.N:]

    def __getitem__(self, index):
        adjusted_idx = index - self.len + len(self.list)

        if index < 0:
            raise ValueError("TimeStore does not support negative indexing")
        if index >= self.len:
            raise ValueError("TimeStore index {} is too large".format(index))
        if adjusted_idx < 0:
            raise ValueError("TimeStore attempting to access deleted entry")

        return self.list[adjusted_idx]

In [7]:
def gaussian_posterior(y, prior_mean, prior_cov, G, V, G_fn=None):
    # Compute posterior mean, cov of N(x; prior_mean, prior_cov) N(y; G @ x, V)
    # y: (*, ydim), prior_mean: (*, xdim), prior_cov: (*, xdim, xdim),
    # G: (*, ydim, xdim), V: (*, ydim, ydim)
    K = prior_cov @ G.transpose(-2, -1) @ torch.linalg.inv(G @ prior_cov @ G.transpose(-2, -1) + V)
    I_KG = torch.eye(prior_cov.shape[-1], device=prior_cov.device) - K @ G
    post_cov = I_KG @ prior_cov
    if G_fn is None:
        post_mean = (prior_mean.unsqueeze(-2) @ I_KG.transpose(-2, -1) +
                     y.unsqueeze(-2) @ K.transpose(-2, -1)).squeeze(-2)
    else:
        post_mean = prior_mean + ((y - G_fn(prior_mean)).unsqueeze(-2) @ K.transpose(-2, -1)).squeeze(-2)
    return post_mean, post_cov

In [8]:
def sample_cov(x, y=None, w=None):
    assert len(x.shape) == 2
    if w is None:
        w = 1 / x.shape[0] * torch.ones((x.shape[0], 1)).to(x.device)
    else:
        w = w.view(x.shape[0], 1)
    x_centred = x - (x * w).sum(0)
    if y is None:
        y_centred = x_centred
    else:
        y_centred = y - (y * w).sum(0)
    cov = (w * x_centred).t() @ y_centred / (1 - (w**2).sum())
    return cov

In [9]:
import Utils as utils
def estimate_joint_kl(model, num_samples, kalman_mean_T, kalman_cov_T,
                        kalman_mean_Tm1, kalman_cov_Tm1, true_F, true_U):
        """
            Estimates KL (q(x_{t-1}, x_t | y_{1:t}) || p(x_{t-1}, x_t | y_{1:t}))
            using num_samples
        """
        with torch.no_grad():
            x_samples, all_q_stats = model.sample_joint_q_t(num_samples, 1)
            x_Tm1, x_T = x_samples
            log_q_t = model.compute_log_q_t(x_T, *all_q_stats[1])
            log_q_t_1 = model.compute_log_q_t(x_Tm1, *all_q_stats[0])

            log_p_x = utils.back_1_joint_smoothing_prob(x_T, x_Tm1,
                                                        kalman_mean_T, kalman_cov_T,
                                                        kalman_mean_Tm1, kalman_cov_Tm1,
                                                        true_F, true_U)

            return torch.mean(log_q_t + log_q_t_1 - log_p_x)


### $\underline{Data\text{ }Generation:}$

In [10]:
DIM = cfg.data.dim

if not cfg.data.diagFG:
    raise NotImplementedError

if cfg.data.path_to_data is None:
    F, G, U, V = DG.construct_HMM_matrices(dim=DIM,
                                        F_eigvals=np.random.uniform(
                                            cfg.data.F_min_eigval,
                                            cfg.data.F_max_eigval, (DIM)),
                                        G_eigvals=np.random.uniform(
                                            cfg.data.G_min_eigval,
                                            cfg.data.G_max_eigval, (DIM)),
                                        U_std=cfg.data.U_std,
                                        V_std=cfg.data.V_std,
                                        diag=cfg.data.diagFG)

    data_gen = DG.GaussianHMM(xdim=DIM, ydim=DIM, F=F, G=G, U=U, V=V)
    x_np, y_np = data_gen.generate_data(cfg.data.num_data)

    save_np('datapoints.npy', np.stack((x_np, y_np)))
    save_np('F.npy', F)
    save_np('G.npy', G)
    save_np('U.npy', U)
    save_np('V.npy', V)
else:
    path_to_data = hydra.utils.to_absolute_path(cfg.data.path_to_data) + '/'
    F, G, U, V = np.load(path_to_data + 'F.npy'), \
                np.load(path_to_data + 'G.npy'), \
                np.load(path_to_data + 'U.npy'), \
                np.load(path_to_data + 'V.npy')
    xystack = np.load(path_to_data + 'datapoints.npy')
    x_np = xystack[0, :, :]
    y_np = xystack[1, :, :]

print("True F: ", F)
print("True G: ", G)

True F:  [[0.979997   0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.76476305 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.53951532 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.90432027 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.86441557 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.88183115
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.92812136 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.57421853 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.69921678 0.        ]
 [0.         0.     

#### $\underline{Kalman-Filter:}$

In [11]:
def analytic_kalman_phi_update(model, T, G, F, U, V, y_T):
    '''
    xtp1 = Fxtp + w, w sim N(0,U)
    yt = Gxt + v, w sim N(0,V)

    Pp: cov of the nonadapted error
    K: Kalman gain
    z: the innovation
    new_mean: adapted x
    new_cov: new error covariance
    cond_cov: 
    '''
    
    prev_mean = model.q_t_mean_list[T-1].reshape(model.xdim, 1)
    prev_cov = torch.diag(torch.exp(2*model.q_t_log_std_list[T-1]))
    y_T = y_T.reshape(model.ydim, 1)

    xp = F @ prev_mean
    Pp = F @ prev_cov @ F.T + U
    S = G @ Pp @ G.T + V
    K = Pp @ G.T @ torch.inverse(S)
    z = y_T - G @ xp

    new_mean = xp + K @ z
    new_cov = (torch.eye(model.xdim).to(device) - K @ G) @ Pp

    cond_cov = torch.inverse(torch.inverse(prev_cov) + \
        F.T @ torch.inverse(U) @ F) 
    cond_weight = cond_cov @ F.T @ torch.inverse(U)
    cond_bias = cond_cov @ torch.inverse(prev_cov) @ prev_mean

    model.q_t_mean_list[T].data = new_mean.reshape(model.xdim)
    model.q_t_log_std_list[T].data = 0.5 * torch.log(torch.diag(new_cov))
    model.cond_q_t_mean_net_list[T].weight.data = cond_weight
    model.cond_q_t_mean_net_list[T].bias.data = cond_bias.reshape(model.xdim)
    model.cond_q_t_log_std_list[T].data = 0.5 * torch.log(torch.diag(cond_cov))

In [12]:
kalman_xs = np.zeros((y_np.shape[0], DIM))
kalman_Ps = np.zeros((y_np.shape[0], DIM, DIM))

# For t=0
kalman_Ps[0, :, :] = np.linalg.inv(np.eye(DIM) + G.T @ np.linalg.inv(V) @ G)
kalman_xs[0, :] = kalman_Ps[0, :, :] @ G.T @ np.linalg.inv(V) @ y_np[0, :]

kalman_filter = KFM.KalmanFilter(x_0=kalman_xs[0, :], P_0=kalman_Ps[0, :, :], F=F, G=G, U=U,
                                    V=V)

for t in range(1, y_np.shape[0]):
    kalman_filter.update(y_np[t, :])
    kalman_xs[t, :] = kalman_filter.x
    kalman_Ps[t, :, :] = kalman_filter.P
kalman_xs_pyt = torch.from_numpy(kalman_xs).float()
kalman_Ps_pyt = torch.from_numpy(kalman_Ps).float()

F_init, G_init, _, _ = DG.construct_HMM_matrices(dim=DIM,
                                        F_eigvals=np.random.uniform(
                                            cfg.data.F_min_eigval,
                                            cfg.data.F_max_eigval, (DIM)),
                                        G_eigvals=np.random.uniform(
                                            cfg.data.G_min_eigval,
                                            cfg.data.G_max_eigval, (DIM)),
                                        U_std=cfg.data.U_std,
                                        V_std=cfg.data.V_std,
                                        diag=cfg.data.diagFG)

y = torch.from_numpy(y_np).float().to(device)

F = torch.from_numpy(F).float().to(device)
G = torch.from_numpy(G).float().to(device)

F_init = torch.from_numpy(F_init).float().to(device)
G_init = torch.from_numpy(G_init).float().to(device)

U = torch.from_numpy(U).float().to(device)
V = torch.from_numpy(V).float().to(device)

mean_0 = torch.zeros(DIM).to(device)
std_0 = torch.sqrt(torch.diag(U))

In [13]:
print(F_init)
print(F)

tensor([[0.9381, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.9515, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.5977, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.8241, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.7636, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7628, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8681, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7861, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7239,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.7357]])
tensor([[

In [14]:
print(kalman_xs_pyt.shape)

torch.Size([50000, 10])


In [15]:
print(kalman_Ps_pyt.shape)

torch.Size([50000, 10, 10])


### $\underline{Model\text{ }parametrization:}$

In [16]:
# we define the observalbe's transiton map
class F_Module(nn.Module):
        def __init__(self):
            super().__init__()
            self.register_parameter('weight',
                nn.Parameter(torch.zeros(DIM)))
            self.F_mean_fn = lambda x, t: self.weight * x
            self.F_cov_fn = lambda x, t: U
            self.F_cov = U

        def forward(self, x, t=None):
            return Independent(Normal(self.F_mean_fn(x, t),
                torch.sqrt(torch.diag(U))), 1)

# we define the latent's transiton map
class G_Module(nn.Module):
    def __init__(self):
        super().__init__()
        self.register_parameter('weight',
            nn.Parameter(torch.zeros(DIM)))
        self.G_mean_fn = lambda x, t: self.weight * x
        self.G_cov = V

    def forward(self, x, t=None):
        return Independent(Normal(self.G_mean_fn(x, t),
            torch.sqrt(torch.diag(V))), 1)
            
# we define the initialization class
class p_0_dist_module(nn.Module):
    def __init__(self):
        super().__init__()
        self.mean_0 = mean_0
        self.cov_0 = torch.eye(DIM, device=device) * std_0 ** 2

    def forward(self):
        return Independent(Normal(mean_0, std_0), 1)

In [17]:
# we initialize the two linking modules and the initial prior on the latent variable.
F_fn = F_Module().to(device)
G_fn = G_Module().to(device)
p_0_dist = p_0_dist_module().to(device)

#### $\underline{Parameters:}$

In [18]:
# count the number of parameters to be learned
G_theta_dim = sum([p.numel() for p in G_fn.parameters()])
F_theta_dim = sum([p.numel() for p in F_fn.parameters()])
print("G theta dim", G_theta_dim)
print("F theta dim", F_theta_dim)
if cfg.data.matrices_to_learn == 'F':
    theta_dim = F_theta_dim
elif cfg.data.matrices_to_learn == 'G':
    theta_dim = G_theta_dim
elif cfg.data.matrices_to_learn == 'FG':
    theta_dim = F_theta_dim + G_theta_dim
else:
    raise ValueError(cfg.data.matrices_to_learn)
print("Theta dim", theta_dim)

# get the model parameters
def get_model_parameters():
        if cfg.data.matrices_to_learn == 'F':
            return F_fn.parameters()
        elif cfg.data.matrices_to_learn == 'G':
            return G_fn.parameters()
        elif cfg.data.matrices_to_learn == 'FG':
            return [*F_fn.parameters(), *G_fn.parameters()]

G theta dim 10
F theta dim 10
Theta dim 20


#### $\underline{Variational\text{ }Model\text{ }Specifications-(\phi):}$

In [19]:
def cond_q_mean_net_constructor():
        return torch.nn.Linear(DIM, DIM).to(device)

if cfg.phi_training.func_type == 'Vx_t':
    print("Using phi model Vx_t")

    sigma = cfg.phi_training.KRR_sigma
    lam = cfg.phi_training.KRR_lambda
    train_sigma = cfg.phi_training.KRR_train_sigma
    train_lam = cfg.phi_training.KRR_train_lam

    def KRR_constructor():
        return models.KernelRidgeRegressor(models.MaternKernel(
            sigma=sigma, lam=lam, train_sigma=train_sigma, train_lam=train_lam)).to(device)

    phi_model = models.Vx_t_phi_t_Model(
        device, DIM, DIM, torch.randn(DIM, device=device),
        torch.zeros(DIM, device=device), cond_q_mean_net_constructor,
        torch.zeros(DIM, device=device), F_fn, G_fn, p_0_dist,
        cfg.phi_training.phi_t_init_method,
        cfg.phi_training.window_size,
        KRR_constructor, cfg.phi_training.KRR_init_sigma_median,
        cfg.phi_training.approx_decay,
        cfg.phi_training.approx_with_filter,
        max(cfg.phi_training.window_size, cfg.theta_training.window_size)+1
    )

elif cfg.phi_training.func_type == 'analytic':
    print("Using analytic phi updates")
    phi_model = models.NonAmortizedModelBase(
        device, DIM, DIM, torch.zeros(DIM, device=device),
        torch.zeros(DIM, device=device), cond_q_mean_net_constructor,
        torch.zeros(DIM, device=device), F_fn, G_fn, p_0_dist,
        'last', 1, cfg.theta_training.window_size + 1
    )

Using phi model Vx_t


#### $\underline{Real\text{ }Model\text{ }Specifications-(\theta)}:$

In [20]:
matrices_to_learn = cfg.theta_training.matrices_to_learn

def add_theta_grads_to_params(grads):
    if matrices_to_learn == 'G':
        phi_model.G_fn.weight.grad += grads

    elif matrices_to_learn == 'F':
        phi_model.F_fn.weight.grad += grads

    elif matrices_to_learn == 'FG':
        phi_model.F_fn.weight.grad += grads[:int(theta_dim/2)]
        phi_model.G_fn.weight.grad += grads[int(theta_dim/2):]

if cfg.theta_training.func_type == 'neural_net':
    print("Learning theta grad with neural nets")
    h = cfg.theta_training.net_hidden_dim

    def theta_func_constructor():
        nnlist = [nn.Linear(DIM, h), nn.ReLU()]
        for i in range(cfg.theta_training.net_num_hidden_layers-1):
            nnlist += [nn.Linear(h, h), nn.ReLU()]
        nnlist += [nn.Linear(h, theta_dim)]

        return models.NNFuncEstimator(
            nn.Sequential(*nnlist), DIM, theta_dim
        ).to(device)
elif cfg.theta_training.func_type == 'kernel':
    print("Learning theta grads with kernel")
    def theta_func_constructor():
        krr = models.KernelRidgeRegressor(
            models.MaternKernel(
                sigma=cfg.theta_training.KRR_sigma,
                lam=cfg.theta_training.KRR_lambda,
                train_sigma=cfg.theta_training.KRR_train_sigma,
                train_lam=cfg.theta_training.KRR_train_lam
            )
        )
        class KRRWrapper(nn.Module):
            def __init__(self, krr):
                super().__init__()
                self.krr = krr

            def fit(self, x_fit, *fs):
                self.krr.fit(x_fit, *fs)
            def forward(self, x):
                return self.krr.forward(x)[0]
            def update_K(self):
                self.krr.update_K()
            def train(self, mode=True):
                return self.krr.train(mode)

        return KRRWrapper(krr).to(device)
elif cfg.theta_training.func_type == 'JELBO':
    print("Learning theta grads with JELBO")
    def theta_func_constructor():
        class ZeroModule(nn.Module):
            def __init__(self):
                super().__init__()

            def forward(self, x):
                return torch.zeros([x.shape[0], theta_dim]).to(x.device)
        return ZeroModule().to(device)
elif cfg.theta_training.func_type == 'VJF':
    print("Learning theta grads with VJF")
    def theta_func_constructor():
        class ZeroModule(nn.Module):
            def __init__(self):
                super().__init__()

            def forward(self, x):
                return torch.zeros([x.shape[0], theta_dim]).to(x.device)
        return ZeroModule().to(device)
elif cfg.theta_training.func_type == 'analytic_S':
    print("Learning theta grads with analytic S")
    def theta_func_constructor():
        return models.TrueSDiagFG(DIM, y, phi_model.G_fn.weight.data.clone(),
                                    U, V, matrices_to_learn).to(device)
else:
    raise NotImplementedError


if cfg.theta_training.func_type == 'VJF':
    theta_grad = models.ThetaGradVJF(
        device, phi_model, theta_func_constructor,
        cfg.theta_training.window_size, theta_dim, get_model_parameters,
        add_theta_grads_to_params
    )
else:
    theta_grad = models.ThetaGradGaussian(
        device, phi_model, theta_func_constructor,
        cfg.theta_training.window_size, theta_dim, get_model_parameters,
        add_theta_grads_to_params
    )
theta_optim = torch.optim.Adam(get_model_parameters(),
    lr=cfg.theta_training.theta_lr)
if cfg.theta_training.theta_lr_decay_type == 'exponential':
    theta_decay = torch.optim.lr_scheduler.StepLR(theta_optim,
        step_size=1, gamma=np.exp(
            (1/cfg.theta_training.num_steps_theta_lr_oom_drop) * np.log(0.1)))
elif cfg.theta_training.theta_lr_decay_type == 'robbins-monro':
    lr_decay_rate = cfg.theta_training.robbins_monro_theta_lr_decay_rate
    lr_decay_bias = cfg.theta_training.robbins_monro_theta_lr_decay_bias
    theta_decay = torch.optim.lr_scheduler.LambdaLR(theta_optim,
        lr_lambda=lambda epoch: lr_decay_bias / (lr_decay_bias + epoch ** lr_decay_rate))
else:
    raise NotImplementedError

rmle = models.LinearRMLEDiagFG(np.zeros((DIM,1)), np.eye(DIM),
    F_init.detach().cpu().numpy().copy() if 'F' in cfg.theta_training.matrices_to_learn else F.detach().cpu().numpy().copy(),
    G_init.detach().cpu().numpy().copy() if 'G' in cfg.theta_training.matrices_to_learn else G.detach().cpu().numpy().copy(),
    U.cpu().detach().numpy().copy(), V.cpu().detach().numpy().copy(),
    cfg.theta_training.theta_lr,
    cfg.theta_training.matrices_to_learn)

Learning theta grads with kernel


#### $\underline{Training\text{ }Specifications}:$

In [21]:
theta_optim = torch.optim.Adam(get_model_parameters(),
        lr=cfg.theta_training.theta_lr)
if cfg.theta_training.theta_lr_decay_type == 'exponential':
    theta_decay = torch.optim.lr_scheduler.StepLR(theta_optim,
        step_size=1, gamma=np.exp(
            (1/cfg.theta_training.num_steps_theta_lr_oom_drop) * np.log(0.1)))
elif cfg.theta_training.theta_lr_decay_type == 'robbins-monro':
    lr_decay_rate = cfg.theta_training.robbins_monro_theta_lr_decay_rate
    lr_decay_bias = cfg.theta_training.robbins_monro_theta_lr_decay_bias
    theta_decay = torch.optim.lr_scheduler.LambdaLR(theta_optim,
        lr_lambda=lambda epoch: lr_decay_bias / (lr_decay_bias + epoch ** lr_decay_rate))
else:
    raise NotImplementedError

### $\underline{Training:}$

In [22]:
Gs = []
Fs = []
rmle_Gs = []
rmle_Fs = []
joint_kls = []
theta_func_losses = []
times = []
filter_means = []
filter_stds = []

pbar = tqdm.tqdm(range(0, cfg.data.num_data))

for T in pbar:
    start_time = time.time()

    # ---------- Advance timesteps --------------

    phi_model.advance_timestep(y[T, :])
    theta_grad.advance_timestep()


    # ----------- Phi optimization ----------------



    if cfg.phi_training.func_type == 'analytic' and T>0:
        tmpG = torch.diag(phi_model.G_fn.weight.data.clone())
        tmpF = torch.diag(phi_model.F_fn.weight.data.clone())
        analytic_kalman_phi_update(phi_model, T, tmpG, tmpF, U, V, y[T,:])

    elif cfg.phi_training.func_type == 'Vx_t':
        phi_optim = torch.optim.Adam(phi_model.get_phi_T_params(),
            lr=cfg.phi_training.phi_lr)
        phi_decay = torch.optim.lr_scheduler.StepLR(
            phi_optim, 1, cfg.phi_training.phi_lr_decay_gamma
        )
        for i in range(cfg.phi_training.phi_iters):
            phi_optim.zero_grad()
            phi_model.populate_phi_grads(y,
                cfg.phi_training.phi_minibatch_size)
            phi_optim.step()
            phi_decay.step()

        if T >= cfg.phi_training.window_size - 1:
            phi_model.update_V_t(y, cfg.phi_training.V_batch_size)
            Vx_optim = torch.optim.Adam(phi_model.get_V_t_params(),
                lr=cfg.phi_training.V_lr)
            for k in range(cfg.phi_training.V_iters):
                Vx_optim.zero_grad()
                V_loss, _, _ = phi_model.V_t_loss(y,
                    cfg.phi_training.V_minibatch_size)
                V_loss.backward()
                Vx_optim.step()


    # -------------- Theta func training ----------------


    if T >= cfg.theta_training.window_size:
        if cfg.theta_training.func_type == 'neural_net':
            theta_func_optim = torch.optim.Adam(
                theta_grad.get_theta_func_TmL_parameters(),
                lr=cfg.theta_training.net_lr)
            net_inputs, net_targets = theta_grad.generate_training_dataset(
                cfg.theta_training.net_dataset_size, y
            )
            net_inputs = net_inputs.detach()
            net_targets = net_targets.detach()

            theta_grad.theta_func_TmL.update_normalization(
                net_inputs, net_targets, cfg.theta_training.net_norm_decay
            )
            for i in range(cfg.theta_training.net_iters):
                idx = np.random.choice(np.arange(net_inputs.shape[0]),
                    (cfg.theta_training.net_minibatch_size,), replace=False)
                theta_func_optim.zero_grad()
                preds = theta_grad.theta_func_TmL(net_inputs[idx,:])
                loss = torch.mean(
                    torch.sum((preds - net_targets[idx, :])**2, dim=1)
                )
                loss.backward()
                theta_func_optim.step()
                theta_func_losses.append(loss.item())
        elif cfg.theta_training.func_type == 'kernel':
            kernel_inputs, kernel_targets = theta_grad.generate_training_dataset(
                cfg.theta_training.kernel_batch_size, y
            )
            kernel_inputs = kernel_inputs.detach()
            kernel_targets = kernel_targets.detach()
            if T == cfg.theta_training.window_size and \
                cfg.theta_training.KRR_init_sigma_median:
                theta_grad.theta_func_TmL.krr.kernel.log_sigma.data = \
                    torch.tensor(
                        np.log(utils.estimate_median_distance(kernel_inputs)\
                            .astype(float))
                    ).to(device)
                print("Update bandwidth to ", theta_grad.theta_func_TmL.krr.kernel.log_sigma.exp().item())
            theta_grad.theta_func_TmL.fit(kernel_inputs, kernel_targets)

            kernel_optim = torch.optim.Adam(
                theta_grad.theta_func_TmL.parameters(),
                lr=cfg.theta_training.train_kernel_lr
            )
            # Generate new data to train hyperparams on
            if cfg.theta_training.KRR_train_sigma or cfg.theta_training.KRR_train_lam:
                kernel_inputs, kernel_targets = theta_grad.generate_training_dataset(
                    cfg.theta_training.train_kernel_dataset_size, y
                )
                kernel_inputs = kernel_inputs.detach()
                kernel_targets = kernel_targets.detach()
                for i in range(cfg.theta_training.train_kernel_iters):
                    idx = np.random.choice(np.arange(kernel_inputs.shape[0]),
                        (cfg.theta_training.train_kernel_minibatch_size,), replace=False)
                    kernel_optim.zero_grad()
                    preds = theta_grad.theta_func_TmL(kernel_inputs[idx,:])
                    loss = torch.mean(
                        torch.sum((preds - kernel_targets[idx,:])**2, dim=1)
                    )
                    loss.backward()
                    kernel_optim.step()
        elif cfg.theta_training.func_type == 'analytic_S':
            # Compute S_{T-window_size} (T-window_size>0)
            if T > cfg.theta_training.window_size:
                theta_grad.theta_func_TmL.advance_timestep(
                    y[T - cfg.theta_training.window_size],
                    phi_model.F_fn.weight.data.clone(),
                    phi_model.G_fn.weight.data.clone(),
                    qW=phi_model.cond_q_t_mean_net_list[T - cfg.theta_training.window_size].weight.data.clone(),
                    qb=phi_model.cond_q_t_mean_net_list[T - cfg.theta_training.window_size].bias.data.clone(),
                    qcov_diag=torch.exp(2 * phi_model.cond_q_t_log_std_list[T - cfg.theta_training.window_size])
                )



    # ---------------- Theta update ----------------


    if T > cfg.theta_training.theta_updates_start_T:
        theta_optim.zero_grad() 
        theta_grad.populate_theta_grads(
            cfg.theta_training.theta_minibatch_size, y)
        theta_optim.step()
        Gs.append(G_fn.weight.clone().detach().numpy())
        Fs.append(F_fn.weight.clone().detach().numpy())

        pbar.set_postfix({"F MAE": np.mean(np.abs(Fs[-1] - np.diag(np.array(F)))),
                            "G MAE": np.mean(np.abs(Gs[-1] - np.diag(np.array(G))))})

        rmle.step_size = theta_decay.state_dict()['_last_lr'][0]
        rmle.advance_timestep(y[T, :].detach().numpy().copy().reshape((DIM,1)))
        rmle_Gs.append(rmle.G.copy())
        rmle_Fs.append(rmle.F.copy())

        theta_decay.step()


    # -------------- Logging --------------------


    filter_means.append(phi_model.q_t_mean_list[T].detach().cpu().numpy())
    filter_stds.append(phi_model.q_t_log_std_list[T].detach().cpu().numpy())

    if T>0:
        joint_kls.append(estimate_joint_kl(phi_model, 256,
                    kalman_xs_pyt[T, :], kalman_Ps_pyt[T, :, :],
                    kalman_xs_pyt[T - 1, :], kalman_Ps_pyt[T - 1, :, :],
                    F, U).item())

    if (T % (round(max(cfg.data.num_data, cfg.theta_training.num_times_save_data)\
        / cfg.theta_training.num_times_save_data)) == 0) or\
        (T == cfg.data.num_data - 1):

        save_np('Gs.npy', np.array(Gs))
        save_np('Fs.npy', np.array(Fs))
        save_np('rmle_Gs.npy', np.array(rmle_Gs))
        save_np('rmle_Fs.npy', np.array(rmle_Fs))
        save_np('joint_kls.npy', np.array(joint_kls))
        save_np('theta_func_losses.npy', np.array(theta_func_losses))
        save_np('times.npy', np.array(times))
        save_np('filter_means.npy', np.array(filter_means))
        save_np('filter_stds.npy', np.array(filter_stds))
        if cfg.save_models:
            torch.save(phi_model.state_dict(), saved_models_folder_name + \
                '/phi_model_{}.pt'.format(T))
            torch.save(theta_grad.theta_func_TmL.state_dict(),
                saved_models_folder_name + '/theta_model_{}.pt'.format(T))
            torch.save(theta_optim.state_dict(), saved_models_folder_name +\
                '/theta_optim_{}.pt'.format(T))
            torch.save(theta_decay.state_dict(), saved_models_folder_name +\
                '/theta_decay_{}.pt'.format(T))

    times.append(time.time()-start_time)


f, (ax1, ax2) = plt.subplots(1, 2)
rmle_F_maes = np.mean(np.abs(np.diagonal(rmle_Fs, axis1=1, axis2=2) - np.diag(F)), 1)
F_maes = np.mean(np.abs(Fs - np.diag(F)), 1)
rmle_G_maes = np.mean(np.abs(np.diagonal(rmle_Gs, axis1=1, axis2=2) - np.diag(G)), 1)
G_maes = np.mean(np.abs(Gs - np.diag(G)), 1)
ax1.plot(rmle_F_maes)
ax1.plot(F_maes)
ax2.plot(rmle_G_maes)
ax2.plot(G_maes)
plt.show()

print("F RMLE: ", rmle.F.copy())
print("G RMLE: ", rmle.G.copy())
print("F: ", Fs[-1])
print("G: ", Gs[-1])

  0%|          | 0/50000 [00:00<?, ?it/s]

Update bandwidth to  1.9626816157537048
Update bandwidth to  4.460809225987467


ValueError: Expected parameter loc (Tensor of shape (512, 10)) of distribution Normal(loc: torch.Size([512, 10]), scale: torch.Size([512, 10])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<MulBackward0>)